# Traffy Fondue Problem Report Classification

## Setup

In [1]:
# !pip install pandas torch torchtext torchmetrics pythainlp pytorch_lightning wandb scikit-learn

In [2]:
%env WANDB_NOTEBOOK_NAME="traffy-fondue-classification"

env: WANDB_NOTEBOOK_NAME="traffy-fondue-classification"


In [3]:
# Logger
import wandb
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find "traffy-fondue-classification".
wandb: Currently logged in as: noppakorn (meen). Use `wandb login --relogin` to force relogin


True

In [4]:
import pandas as pd

import sklearn
import sklearn.model_selection
import torchmetrics

from pythainlp.tokenize import word_tokenize

import torch
import pytorch_lightning as pl


from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import get_tokenizer

from torch import optim, nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger

seed_everything(42, workers=True)

Global seed set to 42


42

## Dataset

In [5]:
df = pd.read_csv("./traffy-data-2023.csv")
df = df.loc[:, ["type", "comment"]]

df = df.rename({"comment": "text", "type": "tag"}, axis=1)

# Data cleansing

# Clean the tag
def clean_tag(tag: str) -> str:
    tag = tag[1:-1]
    tag = tag.split(",")

    if len(tag) == 0:
        return "Unknown"

    if tag[0] == "ร้องเรียน" and len(tag) > 1:
        return tag[1]
    return tag[0]

df.loc[:, "tag"] = df.loc[:, "tag"].apply(clean_tag)
df = df[~(df.loc[:, "tag"].isin({"Unknown", "", "ร้องเรียน"}))]

# Use only tag with >= 1000 reports
tags = df.loc[:, "tag"].value_counts()
tags = tags[(tags.values >= 1000)]
df = df[df.loc[:, "tag"].isin(tags.keys())]

def clean_text(text):
    return word_tokenize(text, engine="newmm")

df.loc[:, "text"] = df.loc[:, "text"].apply(clean_text)

df = df.reset_index(drop=True)

df

,tag,text
0,ถนน,"[มี, น้ำซึม, ออกมา, บริเวณ, ถนน, , ทำให้, มี,..."
1,ท่อระบายน้ำ,"[ท่อน้ำ, แตก, , น้ำ, ผุด, ไหล, ตลอด]"
2,ท่อระบายน้ำ,"[ฝ่า, ท่อ, ชำรุด]"
3,ท่อระบายน้ำ,"[ฝา, ท่อ, ชำรุด]"
4,จราจร,"[ตำรวจ, ไม่, จับ, มอ, ไซ, ย้อนศร, , วิน, มอ, ..."
...,...,...
67177,ความสะอาด,"[ช่วย, เก็บ, ขยะ, หน่อย, ครับ]"
67178,ถนน,"[ศูนย์, เรื่องราว, ร้องทุกข์, , ได้รับ, การ, ..."
67179,ทางเท้า,"[มี, การ, ขาย, น้ำ, กระท่อม, , และ, กัญชา, เป..."
67180,ถนน,"[ปัญหา, , :, , :, , ประชาชน, ต้องการ, ให้, ..."


In [6]:
print("Tags Statistics")
print(df.tag.value_counts())


print("---")
print("Text Statistics")
print("Max words per sentence", df.text.apply(len).max())
print("Average words per sentence", df.text.apply(len).mean())

Tags Statistics
tag
ถนน            24471
ทางเท้า         7750
ความสะอาด       5983
แสงสว่าง        5108
กีดขวาง         4269
ความปลอดภัย     3404
ป้าย            2863
เสียงรบกวน      2705
ท่อระบายน้ำ     2547
จราจร           2073
ต้นไม้          1933
สะพาน           1823
สัตว์จรจัด      1201
สายไฟ           1052
Name: count, dtype: int64
---
Text Statistics
Max words per sentence 2492
Average words per sentence 57.73838230478402


In [7]:
def yield_tokens(sentences_tokenized):
    for word in sentences_tokenized:
        yield word


vocab = build_vocab_from_iterator(yield_tokens(df.text.tolist()), specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [8]:
tag_to_idx = {j:i for i, j in enumerate(tags.keys())}
idx_to_tag = {j:i for i, j in tag_to_idx.items()}

In [9]:
class TraffyDataset(Dataset):
    def __init__(self, sentences_tokenized, tag, vocab, tag_to_idx, pad_length=256):
        self.sentences_tokenized = sentences_tokenized
        self.tag = tag
        self.vocab = vocab
        self.tag_to_idx = tag_to_idx

        self.pad_length = pad_length
        self.padding_index = self.vocab["<pad>"]

    def __len__(self):
        return len(self.sentences_tokenized)

    def __getitem__(self, idx):
        tokens = self.vocab(self.sentences_tokenized[idx])
        sentence_encoded = torch.LongTensor(tokens[:self.pad_length])

        pad = nn.ConstantPad1d(
            (0, self.pad_length - len(sentence_encoded)), self.padding_index)
        sentence_encoded = pad(sentence_encoded)

        tag_encoded = torch.tensor(tag_to_idx[self.tag[idx]])
        return sentence_encoded, tag_encoded

In [10]:
class TraffyModel(pl.LightningModule):
    def __init__(self, vocab_len, class_count, config):
        super().__init__()
        self.config = config

        self.embedding = nn.Embedding(
            vocab_len, self.config["embedding_dim"])

        # self.dropout = nn.Dropout(p=self.config["dropout_rate"])
        self.lstm = nn.LSTM(
            self.config["embedding_dim"], self.config["lstm_hidden_size"], batch_first=True)

        self.classifier = nn.Linear(
            self.config["lstm_hidden_size"], class_count)

        self.criterion = nn.CrossEntropyLoss()

        self.accuracy = torchmetrics.Accuracy(
            task="multiclass", num_classes=class_count)
        self.f1 = torchmetrics.F1Score(
            task="multiclass", average="macro", num_classes=class_count)

    def training_step(self, batch, batch_idx):
        tokens, labels = batch
        x = self.embedding(tokens)
        _, (x, _) = self.lstm(x)
        x = torch.flatten(x, start_dim=0, end_dim=1)
        x = self.classifier(x)

        loss = self.criterion(x, labels)

        self.log("loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        tokens, labels = batch
        x = self.embedding(tokens)
        _, (x, _) = self.lstm(x)
        x = torch.flatten(x, start_dim=0, end_dim=1)
        x = self.classifier(x)

        val_loss = self.criterion(x, labels)

        self.log("val_loss", val_loss)

        val_acc = self.accuracy(x, labels)
        self.log('val_acc', val_acc, on_epoch=True)
        val_f1 = self.f1(x, labels)
        self.log('val_f1', val_f1, on_epoch=True)

    def forward(self, tokens):
        x = self.embedding(tokens)
        _, (x, _) = self.lstm(x)
        x = torch.flatten(x, start_dim=0, end_dim=1)
        x = self.classifier(x)

        return x

    def configure_optimizers(self):
        optimizer = optim.Adam(
            self.parameters(), lr=self.config["lr"])
        return optimizer

In [11]:
config = {
    "epochs": 10,
    "lr": 0.001,
    "embedding_dim": 300,
    "lstm_hidden_size": 256,
    "batch_size": 128,
}

In [12]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df.loc[:, "text"].tolist(), df.loc[:, "tag"].tolist(), test_size=0.2, random_state=42)

In [13]:
train_dataset = TraffyDataset(X_train, y_train, vocab, tag_to_idx)
val_dataset = TraffyDataset(X_test, y_test, vocab, tag_to_idx)
train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"])
val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"])

In [14]:
model = TraffyModel(vocab_len=len(vocab), class_count=len(tag_to_idx), config=config)

In [15]:
wandb_logger = WandbLogger(project="traffy-fondue-classification")

In [16]:
# Normal Training
# trainer = Trainer(
#     logger=logger, min_epochs=config["epochs"], max_epochs=config["epochs"])

# Training using gpu
trainer = Trainer(accelerator="gpu", devices=1, deterministic=True, logger=wandb_logger,
                  min_epochs=config["epochs"], max_epochs=config["epochs"])



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params
--------------------------------------------------
0 | embedding  | Embedding          | 9.6 M 
1 | lstm       | LSTM               | 571 K 
2 | classifier | Linear             | 3.6 K 
3 | criterion  | CrossEntropyLoss   | 0     
4 | accuracy   | MulticlassAccuracy | 0     
5 | f1         | MulticlassF1Score  | 0     
--------------------------------------------------
10.2 M    Trainable params
0         Non-trainable params
10.2 M    Total params
40.665    Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  5.92it/s]

/home/noppakorn/.virtualenvs/neural-project/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/noppakorn/.virtualenvs/neural-project/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 9: 100%|██████████| 420/420 [01:03<00:00,  6.59it/s, v_num=klur]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 420/420 [01:04<00:00,  6.54it/s, v_num=klur]


In [18]:
print("Running the model on the validation set after the last epoch")
res = trainer.validate(model, dataloaders=val_dataloader)
print(f"Accuracy on validation set: {res[0]['val_acc']:.04f}")
print(f"Accuracy on validation set: {res[0]['val_f1']:.04f}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Running the model on the validation set after the last epoch
Validation DataLoader 0: 100%|██████████| 105/105 [00:06<00:00, 17.14it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Runningstage.validating metric      DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.7844012975692749
         val_f1             0.7100043892860413
        val_loss            0.6673240065574646
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Accuracy on validation set: 0.7844
Accuracy on validation set: 0.7100
